### Landsat data methodology for post-fire recovery analysis in Lebanon

This notebook outlines the methodology used to retrieve Landsat imagery via GEE. It is based on the Spectral Trend Database [repository](https://github.com/SchmidtDSE/spectral_trend_database), created by Brookie Guzder-Williams

This notebook queries Landsat 5, 7, and 8 and returns monthly values of relevant bands as well as the vegetation indices *NDVI* and *EVI*. This code expected a monthly time span and returns a geoTIFF for each band for that months. This notebook is designed to test the code, which then is scaled to retrieve the full monthly timeseries. 

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='dse-staff')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

In [2]:
ee.Authenticate()

EEException: Cannot authenticate: Invalid request.

In [ ]:
ee.Initialize(project='dse-staff')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

$\downarrow$ This code is basically taken from the spectral database repository [(`gee/landsat.py`)](https://github.com/SchmidtDSE/spectral_trend_database/blob/b200a3d0faf236cef8677a728ae693af94296192/spectral_trend_database/gee/landsat.py#L4) with only minor changes 

In [ ]:
#Global constants

NOMINAL_SCALE = 30
GRID_DEGREE_SIZE = 0.0439453125
LSAT_SCALE_FACTOR = 0.0000275
LSAT_OFFSET = -0.2
TRANSFORM = [GRID_DEGREE_SIZE, 0, 0, 0, -GRID_DEGREE_SIZE, 0]
MASK_VALUE = 2.1474836e9
MAX_ERR = 1
LANDSAT_BUFFER_RADIUS = 0
EE_CRS = 'EPSG:3857'
MASK_VALUES = [2.1474836e9, 0, 2**16 - 1]
MASK_VALUE_BAND = 'blue'
QA_BANDS = ['QA_PIXEL', 'QA_RADSAT']
L8_SR_ID = 'LANDSAT/LC08/C02/T1_L2'
L7_SR_ID = 'LANDSAT/LE07/C02/T1_L2'
L5_SR_ID = 'LANDSAT/LT05/C02/T1_L2'
L8_BANDS = [
    'SR_B2',
    'SR_B3',
    'SR_B4',
    'SR_B5',
    'SR_B6',
    'SR_B7'
]
L57_BANDS = [
    'SR_B1',
    'SR_B2',
    'SR_B3',
    'SR_B4',
    'SR_B5',
    'SR_B7'
]
HARMONIZED_BANDS = [
    'blue',
    'green',
    'red',
    'nir',
    'swir1',
    'swir2'
]
MISSIONS: dict[int, dict] = {
    8: {
        'id': L8_SR_ID,
        'bands': L8_BANDS,
        "dates": "2013-03-18T15:58:14 -"
    },
    7: {
        'id': L7_SR_ID,
        'bands': L57_BANDS,
        "dates": "1999-05-28T01:02:17 -"
    },
    5: {
        'id': L5_SR_ID,
        'bands': L57_BANDS,
        'dates': "1984-03-16T16:18:01 - 2012-05-05T17:54:06"
    }
}

`qa_mask = _im.select('QA_PIXEL').bitwiseAnd(0b11111).eq(0)` checks if first 5 bits of qa_pixel are 0, if yes, set pixel to 1
this creates a binary mask to mask for clouds. 
See page 13 on [Landsat 8 documentation](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat8-9-Collection2-Level2-Science-Product-Guide-v6.pdf) and page 14 on [Landsat 5 documentation](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1618_Landsat-4-7_C2-L2-ScienceProductGuide-v4.pdf)

In [ ]:
def cloud_masked_rescaled_image(
        im: ee.Image,
        bands: list[str] = HARMONIZED_BANDS,
        mission: Optional[int] = None) -> ee.Image:
    """ cloud mask/rescaled landsat image
    Args:
        im (ee.Image): landsat image
        bands (list[str]): optical bands to be kept
        mission (Optional[int]):
            one of 8, 7, 5 (see `MISSIONS` dict above)
            if exists adds `mission` property to ee.image

    Returns:
         (ee.Image) cloud masked, scaled and offset, bands
         maintaining im properties and timestamps.
    """
    _im = ee.Image(im)
    qa_mask = _im.select('QA_PIXEL').bitwiseAnd(0b11111).eq(0) 
    saturation_mask = _im.select('QA_RADSAT').eq(0)
    _im = _im.updateMask(qa_mask).updateMask(saturation_mask)
    _im = _im.select(bands).multiply(LSAT_SCALE_FACTOR).add(LSAT_OFFSET)
    if mission:
        _im = _im.set('mission', mission)
    _im = _im.set('system:time_start', im.date().millis())

    return ee.Image(_im)

In [ ]:
def cloud_masked_rescaled_ic_for_mission( #inputs native naming, outputs ic with harmonized naming
        mission: int,
        bands: list[str] = HARMONIZED_BANDS,
        data_filter: Optional[ee.Filter] = None) -> ee.ImageCollection:
    """ cloud masked rescaled landsat bands ic for mission
    Args:
        mission (int): one of 8, 7, 5 (see `MISSIONS` dict above)
        data_filter (Optional[ee.Filter]): if exists, filter ic

    Returns:
         (ee.ImageCollection) of cloud masked, scaled and offset, landsat images
    """
    info = MISSIONS[mission]
    ic = ee.ImageCollection(info['id'])

    if data_filter:
        ic = ic.filter(data_filter)
    if bands:
        ic = ic.select(QA_BANDS + info['bands'], QA_BANDS + bands) #rename mission bands to harmonized bands
        # suggestion: wrap this in a function called rename_bands()
    else:
        bands = info['bands']
    ic = ic.map(lambda im: cloud_masked_rescaled_image(im, bands=bands, mission=mission))

    return ic

In [ ]:
def harmonized_cloud_masked_rescaled_ic(
        missions: list[int] = [5, 7, 8],
        data_filter: Optional[ee.Filter] = None) -> ee.ImageCollection:
    """ cloud masked rescaled landsat bands ic for mission
    Args:
        mission (int): one of 8, 7, 5 (see `MISSIONS` dict above)
        data_filter (Optional[ee.Filter]): if exists, filter ic

    Returns:
         (ee.ImageCollection) of cloud masked, scaled and offset, landsat images
    """
    ic = cloud_masked_rescaled_ic_for_mission(missions[0], data_filter=data_filter)
    for m in missions[1:]:
        ic = ic.merge(cloud_masked_rescaled_ic_for_mission(m, data_filter=data_filter))

    return ic

In the spectral database repository, the following code is executed in script [export_landsat_data.py](https://github.com/SchmidtDSE/spectral_trend_database/blob/a4fe0df7822d71ffbf95f0ce1aa485cad7dfba93/scripts/step-2.export_landsat_data.py#L44) which imports the functions defined above. What it does it it grabs the whole Landsat archiv for a pixel, indicated by (lon, lat) and returns it as an xarray object. 

However, in our case we want to grad data for a polygon and return the monthly mean as a `.tiff` So from here on the code diverges more.

$\downarrow$ We want to calculate the indices in the cloud


- `evi`: 2.5 *  (nir + (6 * red) - (7.5 * blue) + 1)

- `ndvi`: (nir - red) / (nir + red)

In [ ]:
def add_indices_to_image(image: ee.Image):
    ''' calculates NDVI and EVI and adds them as bands to the image'''
    
    EVI = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR' : image.select('nir'),
        'RED' : image.select('red'),
        'BLUE': image.select('blue')}).rename('EVI')
    
    NDVI = image.expression('(NIR - RED) / (NIR + RED)', {
        'NIR' : image.select('nir'),
        'RED' : image.select('red')}).rename('NDVI')

    image = image.addBands([EVI, NDVI])

    print('All band names:', image.bandNames().getInfo())

    return image

$\downarrow$ We adapted Brookie's pixel-based function to take a polygon instead of a point as the spatial filter. 

In [ ]:
def get_polygon_monthly_mean_perpixel(
        geom: ee.Geometry.Polygon,
        start_date, end_date,
        show_map = 0) -> ee.Image:

    data_filter = ee.Filter.And(
        ee.Filter.bounds(geom),
        ee.Filter.date(start_date, end_date))
    ic = harmonized_cloud_masked_rescaled_ic(data_filter=data_filter)

    image = ic.mean().clip(geom)

    image = add_indices_to_image(image)

    image = image.set('system:time_start', ee.Date(start_date).millis()) \
                .set('system:time_end', ee.Date(end_date).millis())


    if show_map:
  
        Map = geemap.Map()

        # Define the visualization parameters.
        vizParams = {
          'bands': ['red', 'green', 'blue'],
          'min': 0,
          'max': 0.5,
          'gamma': [0.95, 1.1, 1]
        }
    
    
        Map.addLayer(image, vizParams, "False color composite")
        Map.addLayer(geom, {'color': 'red'}, "Polygon")
        Map.centerObject(geom, 8)
        display(Map)
    
    return image

We test it for a bounding box (scroll to the bottom on more info on where the bouding box is, but not that relevant)

In [ ]:
# Define the coordinates of the rectangle
lon_min = 35.75
lon_max = 36
lat_min = 33.75
lat_max = 34

# Create the rectangle as an Earth Engine Geometry
rectangle = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

image = get_polygon_monthly_mean_perpixel(rectangle, start_date = '2000-01-01', end_date = '2000-01-31', show_map = 1)
image

We next test for for the polygon of Mount Lebanon

In [ ]:
mount_lebanon = ee.FeatureCollection('projects/dse-staff/assets/mount_lebanon')
type(mount_lebanon)

mount_lebanon = mount_lebanon.first().geometry()

image = get_polygon_monthly_mean_perpixel(mount_lebanon, start_date = '2000-01-01', end_date = '2000-01-31', show_map = 1)
image

In [ ]:
def export_image_to_cloud(image: ee.Image, date: str):

    path_file = 'landsat_lebanon/landsat_indices_' + date
    print(path_file)
    
    export_task = ee.batch.Export.image.toCloudStorage(
    image=image,
    description='landsat_export_cog',
    bucket='dse-staff', 
    fileNamePrefix=path_file,  
    fileFormat='GeoTIFF', 
    formatOptions={
        'cloudOptimized': True,  
    },
    maxPixels=1e8,  
    scale=30  
    )

    export_task.start()

    return print(f'{date} saved')
    
export_image_to_cloud(image, 'test_indices_ml')

In [ ]:
import calendar

def generate_year_month_strings(y1, y2):
    result = []
    
    # Step 1: Generate year-month combinations
    for year in range(y1, y2 + 1):
        for month in range(1, 13):
            year_month = f"{year}-{month:02d}"
            result.append(year_month)
    return result

In [ ]:
def generate_first_last_date(year_month):
    year, month = map(int, year_month.split('-'))  # Split the year-month string into year and month
    _, last_day = calendar.monthrange(year, month)  # Get the number of days in the month
    start_date = f"{year}-{month:02d}-01"
    end_date = f"{year}-{month:02d}-{last_day:02d}"

    return start_date, end_date

# Example usage
y1 = 1986
y2 = 2024
year_month_list = generate_year_month_strings(y1, y2)

for m in year_month_list:
    date = generate_first_last_date(m)
    image = get_polygon_monthly_mean_perpixel(mount_lebanon, start_date = date[0], end_date = date[1])
    export_image_to_cloud(image, m)

Not use this list to map over the function and write data out for each month in the list:

(see https://gis.stackexchange.com/questions/343871/filter-image-collection-with-a-month-range-for-each-year-in-google-earth-engine)

In [ ]:
year_month_list

In [ ]:
write_monthly_file(rectangle, '2021-01')

$\downarrow$ Plotting the rectangle I am subsetting: (todo: plot the xarray on top to make sure the selection is correct, can't get that to work as of now due to the different projection systems)

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import Point

# Initialize Earth Engine
ee.Initialize()

# Define the coordinates of the rectangle
lon_min = 35.75
lon_max = 36
lat_min = 33.75
lat_max = 34

# Create the rectangle as an Earth Engine Geometry
rectangle = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

# Convert the Earth Engine geometry to a GeoDataFrame for plotting
polygon = Polygon([
    (lon_min, lat_min),
    (lon_max, lat_min),
    (lon_max, lat_max),
    (lon_min, lat_max),
    (lon_min, lat_min)
])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': [polygon]}, crs="EPSG:4326")

# Define a list of major cities with corrected coordinates
cities = {
    'Beirut': (35.495, 33.8886),  # Corrected: Beirut, Lebanon
    'Damascus': (36.305, 33.513),  # Damascus, Syria
}

# Convert cities to Points for plotting
city_points = [Point(lon, lat) for city, (lon, lat) in cities.items()]
cities_gdf = gpd.GeoDataFrame({'city': list(cities.keys()), 'geometry': city_points}, crs="EPSG:4326")

# Plot the rectangle using Cartopy for basemap
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# Add the natural world map using Cartopy
ax.set_extent([lon_min - 1, lon_max + 1, lat_min - 1, lat_max + 1], crs=ccrs.PlateCarree())
ax.coastlines(resolution='110m')
ax.gridlines(draw_labels=True)

# Add country borders
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='black', linewidth=1)

# Plot the rectangle on the map
gdf.boundary.plot(ax=ax, color='blue', linewidth=2, label='Rectangle')

# Plot cities on the map
cities_gdf.plot(ax=ax, color='red', marker='o', markersize=50, label='Major Cities')

# Add labels for Beirut and Damascus (in English only)
for city, (lon, lat) in cities.items():
    ax.text(lon + 0.05, lat + 0.05, f'{city}', transform=ccrs.PlateCarree(), fontsize=12, color='black', ha='left')

# Set labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Rectangle with Country Borders, Major Cities (English)')

# Add a legend
plt.legend()

# Show the plot
plt.show()
